# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords 
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import ne_chunk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
# !pip install -U spacy
import spacy
from spacy import displacy
from collections import Counter
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_lg
np.random.seed(42)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package words is alrea

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('Message', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df[df.columns[4:]]
# X.head()

### 2. Write a tokenization function to process text data

In [4]:
def tokenize(text):
    '''
    Tokenize text.
    
    INPUT
        text (str): text to be tokenized
    OUTPUT
        tokens (list): list of tokens
    '''
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word not in stopwords.words("english")]
    
    # Reduce words to their root forms
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w).lower().strip() for w in words]
    
    return tokens

# # Sanity check
# tokenize(X[0])

### 3. Build a machine learning pipeline
This machine learning pipeline takes in the `message` column as input and outputs classification results on the other 36 categories in the dataset.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10))),
])

# # View parameters in pipeline
# pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test model
Report the f1 score, precision and recall for each output category of the dataset.

In [8]:
# Make predictions based on the trained model
Y_pred = pipeline.predict(X_test)

In [9]:
# Scores for each category
for i, col in enumerate(Y_test.columns):
    print(f"\nCategory: {col} -----------------------------------")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))


Category: related -----------------------------------
              precision    recall  f1-score   support

           0       0.61      0.36      0.45      2034
           1       0.82      0.93      0.87      6617

   micro avg       0.80      0.80      0.80      8651
   macro avg       0.72      0.64      0.66      8651
weighted avg       0.77      0.80      0.77      8651


Category: request -----------------------------------
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      7201
           1       0.83      0.38      0.52      1450

   micro avg       0.88      0.88      0.88      8651
   macro avg       0.86      0.68      0.73      8651
weighted avg       0.88      0.88      0.86      8651


Category: offer -----------------------------------


C:\Users\angang\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8610
           1       0.00      0.00      0.00        41

   micro avg       1.00      1.00      1.00      8651
   macro avg       0.50      0.50      0.50      8651
weighted avg       0.99      1.00      0.99      8651


Category: aid_related -----------------------------------
              precision    recall  f1-score   support

           0       0.72      0.88      0.79      5044
           1       0.75      0.51      0.61      3607

   micro avg       0.73      0.73      0.73      8651
   macro avg       0.73      0.69      0.70      8651
weighted avg       0.73      0.73      0.71      8651


Category: medical_help -----------------------------------
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      7957
           1       0.66      0.09      0.16       694

   micro avg       0.92      0.92      0.92      8651
   macro 



Category: hospitals -----------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8569
           1       0.00      0.00      0.00        82

   micro avg       0.99      0.99      0.99      8651
   macro avg       0.50      0.50      0.50      8651
weighted avg       0.98      0.99      0.99      8651


Category: shops -----------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8615
           1       0.00      0.00      0.00        36

   micro avg       1.00      1.00      1.00      8651
   macro avg       0.50      0.50      0.50      8651
weighted avg       0.99      1.00      0.99      8651


Category: aid_centers -----------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8543
           1       0.00      0.00      0.00       108

   micro avg

In [10]:
# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Random Forest (default)
---------------------------
Accuracy: 0.23084036527569068
Precision: 0.8112696525300568
Recall: 0.44635458312132004
F1 Score: 0.5758698302541498


### 6. Improve model
Use grid search to find better parameters. 

In [11]:
# Specify parameters for grid search
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
#    'vect__max_df': (0.75, 1.0),
#    'tfidf__use_idf': (True, False),
#    'clf__estimator__n_estimators': [10, 50, 100, 200],
#     'clf__estimator__min_samples_split': [2, 5, 10],
#     'clf__estimator__max_depth': [3, 50, None],
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', cv=3)
cv.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [12]:
# cv.estimator.get_params()

### 7. Test model
Show the accuracy, precision, and recall of the tuned model. We can further improve model performance by running grid search on the full parameter ranges defined above. For the scope of this project, I'm going to move on with this minimal tuning.

In [13]:
# Make predictions based on the trained model
Y_pred = cv.predict(X_test)

In [14]:
# Scores for each category
for i, col in enumerate(Y_test.columns):
    print(f"\nCategory: {col} -----------------------------------")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))


Category: related -----------------------------------
              precision    recall  f1-score   support

           0       0.59      0.36      0.45      2034
           1       0.82      0.92      0.87      6617

   micro avg       0.79      0.79      0.79      8651
   macro avg       0.71      0.64      0.66      8651
weighted avg       0.77      0.79      0.77      8651


Category: request -----------------------------------
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      7201
           1       0.83      0.43      0.56      1450

   micro avg       0.89      0.89      0.89      8651
   macro avg       0.86      0.71      0.75      8651
weighted avg       0.88      0.89      0.87      8651


Category: offer -----------------------------------


C:\Users\angang\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8610
           1       0.00      0.00      0.00        41

   micro avg       1.00      1.00      1.00      8651
   macro avg       0.50      0.50      0.50      8651
weighted avg       0.99      1.00      0.99      8651


Category: aid_related -----------------------------------
              precision    recall  f1-score   support

           0       0.71      0.88      0.79      5044
           1       0.74      0.50      0.60      3607

   micro avg       0.72      0.72      0.72      8651
   macro avg       0.73      0.69      0.69      8651
weighted avg       0.73      0.72      0.71      8651


Category: medical_help -----------------------------------
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      7957
           1       0.64      0.08      0.14       694

   micro avg       0.92      0.92      0.92      8651
   macro 



Category: hospitals -----------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8569
           1       0.00      0.00      0.00        82

   micro avg       0.99      0.99      0.99      8651
   macro avg       0.50      0.50      0.50      8651
weighted avg       0.98      0.99      0.99      8651


Category: shops -----------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8615
           1       0.00      0.00      0.00        36

   micro avg       1.00      1.00      1.00      8651
   macro avg       0.50      0.50      0.50      8651
weighted avg       0.99      1.00      0.99      8651


Category: aid_centers -----------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8543
           1       0.00      0.00      0.00       108

   micro avg

In [15]:
# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Random Forest (tuned)
---------------------------
Accuracy: 0.2311871459946827
Precision: 0.8113650070361187
Recall: 0.4400668750454314
F1 Score: 0.5706341164549802


### 8. Try improving the model further
* try other machine learning algorithms
* try other features besides the TF-IDF

#### Compare different machine learning algorithms

In [16]:
# Random Forest
# Build model pipeline
pipeline_nb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10))),
])

# Train the model
pipeline_nb.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_nb.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Random Forest (default)
---------------------------
Accuracy: 0.22887527453473586
Precision: 0.8084488448844884
Recall: 0.44515519371956097
F1 Score: 0.5741608850553159


In [17]:
# Naive Bayes
# Build model pipeline
pipeline_nb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB())),
])

# Train the model
pipeline_nb.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_nb.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Multinomial Naive Bayes (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

C:\Users\angang\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Multinomial Naive Bayes (default)
---------------------------
Accuracy: 0.16333371864524332
Precision: 0.7817003431185665
Recall: 0.37261030747982843
F1 Score: 0.5046641561446258


In [18]:
# Decision Tree
# Build model pipeline
pipeline_dt = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier())),
])

# Train the model
pipeline_dt.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_dt.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Decision Tree (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Decision Tree (default)
---------------------------
Accuracy: 0.14888452202057564
Precision: 0.6122379474759706
Recall: 0.588027913062441
F1 Score: 0.599888765294772


Overall, Random Forest performs the best among these algorithms, so Random Forest is chosen for further tuning.

#### Add other features besides TF-IDF

In [19]:
# Create custom transformer
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            if len(tokenize(sentence))>0:
                pos_tags = nltk.pos_tag(tokenize(sentence))
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                    return 1
        return 0

    # Fit method
    def fit(self, X, y=None):
        return self

    # Transform method
    def transform(self, X):
        X_tagged = pd.Series(X, name='starting_verb').apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
# # Sanity check
# starting_verb = StartingVerbExtractor()
# starting_verb.transform(X.iloc[0:10])

In [20]:
# Create custom transformer
entity_list = ['TIME', 'PERSON', 'GPE', 'DATE', 'NORP', 'MONEY', 'ORG', 'QUANTITY', 'CARDINAL', 'PERCENT', 
               'LOC', 'PRODUCT', 'FAC', 'ORDINAL', 'WORK_OF_ART', 'LANGUAGE', 'LAW', 'EVENT']
nlp = spacy.load('en_core_web_lg')
class NamedEntityExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def named_entities(self, text):
        doc = nlp(text)
        labels = [ent.label_ for ent in doc.ents]
        labels_count = Counter(labels)
        for ent in entity_list:
            if ent not in labels_count.keys():
                labels_count[ent] = 0
        return labels_count

    # Fit method
    def fit(self, X, y=None):
        return self

    # Transform method
    def transform(self, X):
        X_ent = X.apply(self.named_entities).apply(pd.Series)
        X_ent = X_ent.fillna(0)
        X_ent = X_ent[entity_list]
        return X_ent
    
# # Sanity check
# named_entity = NamedEntityExtractor()
# named_entity.transform(X.iloc[0:10])

In [21]:
# Build model pipeline
pipeline_rf_new = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor()), # this is a custom transformer
        
        ('named_entity', NamedEntityExtractor()) # this is a custom transformer
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10))),
])

In [22]:
# Train the model
pipeline_rf_new.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_rf_new.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Random Forest (default)
---------------------------
Accuracy: 0.24135938041844873
Precision: 0.8101831729276622
Recall: 0.47423130042887257
F1 Score: 0.5982713954927898


Overall, the performance is better than using TF-IDF alone. Therefore, I'm going to use the new pipeline to train the model.

#### Tune the model

In [23]:
# pipeline_rf_new.get_params()

In [24]:
# Specify parameters for grid search
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__vect__max_df': (0.75, 1.0),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
#     'clf__estimator__n_estimators': [10, 50, 100, 200],
#     'clf__estimator__min_samples_split': [2, 5, 10],
#     'clf__estimator__max_depth': [3, 50, None],
#     'features__transformer_weights': (
#         {'text_pipeline': 1, 'starting_verb': 1, 'named_entity': 1},
#         {'text_pipeline': 0.5, 'starting_verb': 1, 'named_entity': 1},
#         {'text_pipeline': 1, 'starting_verb': 0.5, 'named_entity': 0.5},
#     )
}

cv = GridSearchCV(pipeline_rf_new, param_grid=parameters, scoring='accuracy', cv=3)
cv.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [25]:
# cv.estimator.get_params()

We can further improve model performance by running grid search on the full parameter ranges defined above. For the scope of this project, I'm going to move on with this minimal tuning.

#### Evaluate model performance

In [26]:
# Make predictions based on the trained model
Y_pred = cv.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Random Forest (default)
---------------------------
Accuracy: 0.2491041498092706
Precision: 0.8092849715578935
Recall: 0.4808824598386276
F1 Score: 0.603287508834325


### 9. Export model as a pickle file

In [27]:
# Save the best model to disk
filename = 'classifier.pkl'
# pickle.dump(cv, open(filename, 'wb'))

This notebook is then converted to `train_classifier.py`, which runs the steps above to train the classifier based on user inputs.